#Let's do this with raw time series data using **LSTM** (since it is a sequence data)


The objective is that we are trying to predict an integer, from 1 to 6,that represents a human activity. We return a binary representation of every sample objective as a 6 bits vector using One Hot Encoding

In [79]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
import pandas as pd
import numpy as np

In [81]:
#Get the training data
train=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Human_Activity_Recognition/Data/UCI_HAR_Dataset/csv_files/train.csv')

In [82]:
train.tail

<bound method NDFrame.tail of       tBodyAccmeanX  tBodyAccmeanY  ...  Activity      ActivityName
0          0.288585      -0.020294  ...         5          STANDING
1          0.278419      -0.016411  ...         5          STANDING
2          0.279653      -0.019467  ...         5          STANDING
3          0.279174      -0.026201  ...         5          STANDING
4          0.276629      -0.016570  ...         5          STANDING
...             ...            ...  ...       ...               ...
7347       0.299665      -0.057193  ...         2  WALKING_UPSTAIRS
7348       0.273853      -0.007749  ...         2  WALKING_UPSTAIRS
7349       0.273387      -0.017011  ...         2  WALKING_UPSTAIRS
7350       0.289654      -0.018843  ...         2  WALKING_UPSTAIRS
7351       0.351503      -0.012423  ...         2  WALKING_UPSTAIRS

[7352 rows x 564 columns]>

In [83]:
train.describe()

,tBodyAccmeanX,tBodyAccmeanY,tBodyAccmeanZ,tBodyAccstdX,tBodyAccstdY,tBodyAccstdZ,tBodyAccmadX,tBodyAccmadY,tBodyAccmadZ,tBodyAccmaxX,tBodyAccmaxY,tBodyAccmaxZ,tBodyAccminX,tBodyAccminY,tBodyAccminZ,tBodyAccsma,tBodyAccenergyX,tBodyAccenergyY,tBodyAccenergyZ,tBodyAcciqrX,tBodyAcciqrY,tBodyAcciqrZ,tBodyAccentropyX,tBodyAccentropyY,tBodyAccentropyZ,tBodyAccarCoeffX1,tBodyAccarCoeffX2,tBodyAccarCoeffX3,tBodyAccarCoeffX4,tBodyAccarCoeffY1,tBodyAccarCoeffY2,tBodyAccarCoeffY3,tBodyAccarCoeffY4,tBodyAccarCoeffZ1,tBodyAccarCoeffZ2,tBodyAccarCoeffZ3,tBodyAccarCoeffZ4,tBodyAcccorrelationXY,tBodyAcccorrelationXZ,tBodyAcccorrelationYZ,...,fBodyBodyAccJerkMagentropy,fBodyBodyAccJerkMagmaxInds,fBodyBodyAccJerkMagmeanFreq,fBodyBodyAccJerkMagskewness,fBodyBodyAccJerkMagkurtosis,fBodyBodyGyroMagmean,fBodyBodyGyroMagstd,fBodyBodyGyroMagmad,fBodyBodyGyroMagmax,fBodyBodyGyroMagmin,fBodyBodyGyroMagsma,fBodyBodyGyroMagenergy,fBodyBodyGyroMagiqr,fBodyBodyGyroMagentropy,fBodyBodyGyroMagmaxInds,fBodyBodyGyroMagmeanFreq,fBodyBodyGyroMagskewness,fBodyBodyGyroMagkurtosis,fBodyBodyGyroJerkMagmean,fBodyBodyGyroJerkMagstd,fBodyBodyGyroJerkMagmad,fBodyBodyGyroJerkMagmax,fBodyBodyGyroJerkMagmin,fBodyBodyGyroJerkMagsma,fBodyBodyGyroJerkMagenergy,fBodyBodyGyroJerkMagiqr,fBodyBodyGyroJerkMagentropy,fBodyBodyGyroJerkMagmaxInds,fBodyBodyGyroJerkMagmeanFreq,fBodyBodyGyroJerkMagskewness,fBodyBodyGyroJerkMagkurtosis,angletBodyAccMeangravity,angletBodyAccJerkMeangravityMean,angletBodyGyroMeangravityMean,angletBodyGyroJerkMeangravityMean,angleXgravityMean,angleYgravityMean,angleZgravityMean,subject,Activity
count,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,...,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000
mean,0.274488,-0.017695,-0.109141,-0.605438,-0.510938,-0.604754,-0.630512,-0.526907,-0.606150,-0.468604,-0.306043,-0.557121,0.523551,0.387386,0.594374,-0.547569,-0.820041,-0.901874,-0.845784,-0.684345,-0.643770,-0.631069,-0.102993,-0.137937,-0.163946,-0.116599,0.102762,-0.037786,0.130477,-0.026229,0.026322,0.159966,-0.019575,0.009420,0.033291,0.036587,-0.078640,-0.125131,-0.193802,0.105005,...,-0.347657,-0.877995,0.178195,-0.312968,-0.615441,-0.693210,-0.692876,-0.674830,-0.726645,-0.885103,-0.693210,-0.874292,-0.719795,-0.087878,-0.889442,-0.046516,-0.253649,-0.565425,-0.779376,-0.792391,-0.772836,-0.811409,-0.871927,-0.779376,-0.935785,-0.771497,-0.284627,-0.898859,0.125293,-0.307009,-0.625294,0.008684,0.002186,0.008726,-0.005981,-0.489547,0.058593,-0.056515,17.413085,3.643362
std,0.070261,0.040811,0.056635,0.448734,0.502645,0.418687,0.424073,0.485942,0.414122,0.544547,0.282243,0.293867,0.363594,0.343611,0.297818,0.471808,0.259607,0.126333,0.221983,0.371608,0.371581,0.386569,0.468959,0.437268,0.371363,0.306507,0.246593,0.243635,0.230067,0.257383,0.215001,0.208837,0.221432,0.286081,0.216289,0.236226,0.232757,0.363155,0.331122,0.385379,...,0.670112,0.188636,0.253755,0.358631,0.345329,0.335026,0.322850,0.343454,0.293098,0.173574,0.335026,0.192684,0.318706,0.611793,0.157653,0.282665,0.326624,0.326620,0.275733,0.265434,0.287613,0.246680,0.193344,0.275733,0.138683,0.287577,0.630896,0.143135,0.250994,0.321011,0.307584,0

In [84]:
#get the test data
test=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Human_Activity_Recognition/Data/UCI_HAR_Dataset/csv_files/train.csv')

In [85]:
test.head

<bound method NDFrame.head of       tBodyAccmeanX  tBodyAccmeanY  ...  Activity      ActivityName
0          0.288585      -0.020294  ...         5          STANDING
1          0.278419      -0.016411  ...         5          STANDING
2          0.279653      -0.019467  ...         5          STANDING
3          0.279174      -0.026201  ...         5          STANDING
4          0.276629      -0.016570  ...         5          STANDING
...             ...            ...  ...       ...               ...
7347       0.299665      -0.057193  ...         2  WALKING_UPSTAIRS
7348       0.273853      -0.007749  ...         2  WALKING_UPSTAIRS
7349       0.273387      -0.017011  ...         2  WALKING_UPSTAIRS
7350       0.289654      -0.018843  ...         2  WALKING_UPSTAIRS
7351       0.351503      -0.012423  ...         2  WALKING_UPSTAIRS

[7352 rows x 564 columns]>

In [86]:
# Activities are the class labels
# It is a 6 class classification
Labels = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

In [87]:
# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([Labels[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([Labels[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [88]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [89]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

In [90]:
# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/drive/My Drive/Colab Notebooks/Human_Activity_Recognition/Data/UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [91]:

def load_y(subset):
    filename = f'/content/drive/My Drive/Colab Notebooks/Human_Activity_Recognition/Data/UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

In [92]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [93]:
#Importing the tensorflow library and keras
import tensorflow as tf
from tensorflow import keras

In [94]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [95]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [96]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [97]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [98]:
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)
timesteps = len(X_train[0])

In [99]:
input_dim

9

In [100]:
n_classes

6

In [101]:
timesteps

128

In [102]:
#Defining the architecture of LSTM

# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.3))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='relu'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [103]:
# Compiling the model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [104]:
# Training the model
model.fit(X_train,Y_train,batch_size=batch_size,validation_split=0.1,epochs=epochs)

Train on 6616 samples, validate on 736 samples
Epoch 1/30
6616/6616 [==============================] - 90s 14ms/step - loss: 1.8675 - accuracy: 0.3525 - val_loss: 1.0364 - val_accuracy: 0.5231
Epoch 2/30
6616/6616 [==============================] - 89s 13ms/step - loss: 1.3469 - accuracy: 0.4716 - val_loss: 1.0252 - val_accuracy: 0.5027
Epoch 3/30
6616/6616 [==============================] - 88s 13ms/step - loss: 1.4062 - accuracy: 0.4976 - val_loss: 1.0329 - val_accuracy: 0.5679
Epoch 4/30
6616/6616 [==============================] - 88s 13ms/step - loss: 1.0633 - accuracy: 0.6046 - val_loss: 0.7441 - val_accuracy: 0.6345
Epoch 5/30
6616/6616 [==============================] - 89s 13ms/step - loss: 0.8665 - accuracy: 0.6507 - val_loss: 0.6800 - val_accuracy: 0.6454
Epoch 6/30
6616/6616 [==============================] - 89s 13ms/step - loss: 0.8643 - accuracy: 0.6678 - val_loss: 0.7883 - val_accuracy: 0.6250
Epoch 7/30
6616/6616 [==============================] - 90s 14ms/step - loss:

In [105]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 510        0  ...                   0                27
SITTING                178      170  ...                   0                 1
STANDING                10       49  ...                   1                 1
WALKING                  0        1  ...                  19                75
WALKING_DOWNSTAIRS       0        0  ...                 385                 2
WALKING_UPSTAIRS         0        0  ...                   8               317

[6 rows x 6 columns]


In [106]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 2s 631us/step


In [107]:
score

[0.9175840764883861, 0.7583983540534973]